#### CNN for Text Classification

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [2]:

url = R"https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
data = pd.read_csv(url)


In [3]:

print(data.head())

   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation


In [4]:
data = data[["label","tweet"]]
data.columns = ['sentiment', 'text']


tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text'])
X = tokenizer.texts_to_sequences(data['text'])
X = pad_sequences(X,maxlen=100)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['sentiment'])


In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

max_features = len(tokenizer.word_index) + 1
embedding_dim = 50


In [6]:

model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
print(X_train.shape)
print(X_test.shape)

(25569, 100)
(6393, 100)


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           2294500   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           32128     
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 128)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 15, 128)           82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 3, 128)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 384)               0

In [9]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
800/800 [==============================] - 45s 55ms/step - loss: 0.1799 - accuracy: 0.9424 - val_loss: 0.1294 - val_accuracy: 0.9545
Epoch 2/5
800/800 [==============================] - 43s 54ms/step - loss: 0.0664 - accuracy: 0.9772 - val_loss: 0.1293 - val_accuracy: 0.9537
Epoch 3/5
800/800 [==============================] - 44s 55ms/step - loss: 0.0213 - accuracy: 0.9927 - val_loss: 0.1907 - val_accuracy: 0.9537
Epoch 4/5
800/800 [==============================] - 43s 53ms/step - loss: 0.0073 - accuracy: 0.9974 - val_loss: 0.2635 - val_accuracy: 0.9540
Epoch 5/5
800/800 [==============================] - 44s 55ms/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 0.2907 - val_accuracy: 0.9409


In [12]:

input_sentence = input("enter a sentence")

input_seq = tokenizer.texts_to_sequences([input_sentence])
input_seq = pad_sequences(input_seq, maxlen=100)

prediction = model.predict(input_seq)[0]


predicted_sentiment = "Positive" if prediction > 0.5 else "Negative"

print("Predicted sentiment:", predicted_sentiment)

enter a sentenceangry
1/1 [==============================] - 0s 22ms/step
Predicted sentiment: Negative


#### Convolutional Neural Network for Modelling Sentences

In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    "This is a positive sentence.",
    "That was a negative experience.",
    "I feel neutral about this.",
    "The movie was terrible.",
    "I enjoyed the book."
]

labels = np.array([1, 0, 2, 0, 1])


tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(sentences)

In [19]:

max_seq_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

embedding_dim = 100
num_filters = 128
kernel_size = 5

In [20]:

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length))
model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=3, activation='softmax'))  # Output layer with 3 units for 3 classes


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [21]:

model.fit(padded_sequences, labels, epochs=10, batch_size=32, verbose=1)

loss, accuracy = model.evaluate(padded_sequences, labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 1.1014 - accuracy: 0.2000
Epoch 2/10
1/1 [==============================] - 0s 19ms/step - loss: 1.0704 - accuracy: 0.4000
Epoch 3/10
1/1 [==============================] - 0s 20ms/step - loss: 1.0477 - accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 1.0263 - accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 17ms/step - loss: 1.0035 - accuracy: 0.6000
Epoch 6/10
1/1 [==============================] - 0s 22ms/step - loss: 0.9803 - accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 0s 19ms/step - loss: 0.9556 - accuracy: 0.8000
Epoch 8/10
1/1 [==============================] - 0s 19ms/step - loss: 0.9312 - accuracy: 0.8000
Epoch 9/10
1/1 [==============================] - 0s 19ms/step - loss: 0.9054 - accuracy: 0.8000
Epoch 10/10
1/1 [==============================] - 0s 176ms/step - loss: 0.8468 - accuracy: 0.8000
Accuracy: 80.00%


#### Phoneme Detection Using Time Delay Neural Network

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten

X = np.random.randn(1000, 10, 20)
y = np.random.randint(0, 5, size=(1000,))



In [23]:

model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(10, 20)),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
25/25 [==============================] - 2s 22ms/step - loss: 1.6489 - accuracy: 0.1988 - val_loss: 1.6302 - val_accuracy: 0.1650
Epoch 2/10
25/25 [==============================] - 0s 8ms/step - loss: 1.5539 - accuracy: 0.3150 - val_loss: 1.6667 - val_accuracy: 0.1600
Epoch 3/10
25/25 [==============================] - 0s 5ms/step - loss: 1.4840 - accuracy: 0.4013 - val_loss: 1.6702 - val_accuracy: 0.1600
Epoch 4/10
25/25 [==============================] - 0s 5ms/step - loss: 1.4032 - accuracy: 0.4712 - val_loss: 1.6963 - val_accuracy: 0.1600
Epoch 5/10
25/25 [==============================] - 0s 5ms/step - loss: 1.3007 - accuracy: 0.5575 - val_loss: 1.7109 - val_accuracy: 0.1650
Epoch 6/10
25/25 [==============================] - 0s 5ms/step - loss: 1.1841 - accuracy: 0.6137 - val_loss: 1.8544 - val_accuracy: 0.1850
Epoch 7/10
25/25 [==============================] - 0s 5ms/step - loss: 1.0651 - accuracy: 0.6550 - val_loss: 1.9283 - val_accuracy: 0.1650
Epoch 8/10
25/25 [=